## Interpolating from hybrid to height levels

In [1]:
import earthkit.meteo.vertical.array as vertical

#### Hybrid levels

#### Getting the data

First, we get some sample data containing 137 IFS (full) model levels for two points. This data is defined on full model levels in ascending model level number order. So the first level is model level 1 (i.e. the top), while the last one is model level 137 (i.e. the bottom).

In [2]:
from earthkit.meteo.utils.sample import get_sample

DATA = get_sample("vertical_hybrid_data")
sp = DATA.p_surf # surface pressure [Pa]
zs = DATA.z_surf # surface geopotential [m2/s2]
t = DATA.t # temperature [K]
q = DATA.q # specific humidity [kg/kg]

sp.shape, zs.shape, t.shape, q.shape

((2,), (2,), (137, 2), (137, 2))

In [3]:
sp

array([ 95178.337944  , 102659.81019512])

In [4]:
t.shape, t[::40]

((137, 2),
 array([[197.06175232, 198.54808044],
        [230.29292297, 219.00386047],
        [234.56352234, 229.85160828],
        [264.58778381, 292.04481506]]))

In [5]:
A, B = vertical.hybrid_level_parameters(137, model="ifs")
A.shape, B.shape

((138,), (138,))

#### Using interpolate_monotonic

First, compute the geometric height above the ground on all the model levels in the input data. When ``h_reference`` is "ground" the surface geopotential is not used in the computations; to highlight this fact we pass 0 in its position.

In [6]:
# we pass 0 as surface geopotential since it is ignored when h_reference="ground"
h = vertical.height_on_hybrid_levels(t, q, 0, A, B, sp, 
                                     h_type="geometric", 
                                     h_reference="ground")
h.shape, h[::40]

((137, 2),
 array([[81768.8186616 , 80658.19466076],
        [23754.56317888, 24155.41435555],
        [ 9063.31902658, 10239.21253322],
        [  654.59845595,   736.56916746]]))

Next, interpolate the temperature to the target height levels.

In [7]:
target_h=[1000., 5000.] # geometric height, above the ground

t_res = vertical.interpolate_monotonic(t, h, target_h, interpolation="linear")

for hv, rv in zip(target_h, t_res):
    print(hv, rv)

1000.0 [262.3675256  291.44530231]
5000.0 [236.7705579  265.49612184]


Repeat the same computation for specific humidity.

In [8]:
q_res = vertical.interpolate_monotonic(q, h, target_h, interpolation="linear")

for hv, rv in zip(target_h, q_res):
    print(hv, rv)

1000.0 [0.0011256  0.00748044]
5000.0 [7.55046139e-05 1.37700908e-03]


##### Heights above sea level

In [9]:
h_sea = vertical.height_on_hybrid_levels(t, q, zs, A, B, sp, 
                                     h_type="geometric", 
                                     h_reference="sea")

target_h_sea=[1000., 5000.] # geometric height, above the sea

t_res = vertical.interpolate_monotonic(t, h_sea, target_h_sea, interpolation="linear")

for hv, rv in zip(target_h_sea, t_res):
    print(hv, rv)

1000.0 [265.83447529 291.04194846]
5000.0 [239.80992741 264.96290891]


##### Using a subset of levels

We can also use a **subset of levels**. This can significantly speed up the computations and reduce memory usage.

In this case the model level range in the input data must be contiguous and include the bottom-most level. The example below only uses the lowest 50 model levels above the surface.

In [10]:
# compute height on the 50 lowest model levels
# Please note we still need to use the full A and B arrays.
h_sub = vertical.height_on_hybrid_levels(t[-50:], q[-50:], zs, A, B, sp, 
                                     h_type="geometric", 
                                     h_reference="ground")

# interpolate to height levels
t_res = vertical.interpolate_monotonic(t[-50:], h_sub, target_h, interpolation="linear")


for hv, rv in zip(target_h, t_res):
    print(hv, rv)

1000.0 [262.36938948 291.44520344]
5000.0 [236.77461002 265.49528592]


##### Using aux levels

Since, the lowest (full) model level is always above the surface the interpolation fails if the target height is between the surface and the lowest model level. We can easily check the height of the lowest model level in our data:

In [11]:
h[-1]

array([ 9.34342138, 10.21668113])

So the interpolation to 5 m above the ground would not work:

In [12]:
target_h=[5.] # geometric height, above the ground

t_res = vertical.interpolate_monotonic(t, h, target_h, interpolation="linear")

for hv, rv in zip(target_h, t_res):
    print(hv, rv)

5.0 [nan nan]


To overcome this problem we can define "aux" levels with a prescribed input data. As a demonstration, we set the temperature values on the surface in all the input points with the ``aux_min_level_*`` kwargs.

In [13]:
target_h=[5.] # geometric height, above the ground

t_res = vertical.interpolate_monotonic(t, h, target_h, 
                                       aux_min_level_coord=0,
                                       aux_min_level_data=[280., 300.],
                                       interpolation="linear")

for hv, rv in zip(target_h, t_res):
    print(hv, rv)

5.0 [274.04715229 296.50016645]


#### Using interpolate_hybrid_to_height_levels

In [14]:
target_h = [1000., 5000.] # # geometric height, above the ground

t_h = vertical.interpolate_hybrid_to_height_levels(
    t, # data to interpolate
    t, q, 0, A, B, sp, target_h, 
    h_type="geometric", 
    h_reference="ground",
    interpolation="linear")

for vh, th in zip(target_h, t_h):
    print(vh, th)

1000.0 [262.3675256  291.44530231]
5000.0 [236.7705579  265.49612184]


##### Using a subset of levels

It is possible to use only a **subset of the model levels** in the input data. To do so the model level range in the input must be contiguous and include the bottom-most level. The following example only uses the lowest 50 model levels above the surface. 

In [15]:
# using the 50 lowest model levels as input
# Please note we still need to use the full A and B arrays
t_h = vertical.interpolate_hybrid_to_height_levels(
    t[-50:], # data to interpolate
    t[-50:], q[-50:], 0, A, B, sp, target_h, 
    h_type="geometric", 
    h_reference="ground",
    interpolation="linear")

for vh, th in zip(target_h, t_h):
    print(vh, th)

1000.0 [262.3675256  291.44530231]
5000.0 [236.7705579  265.49612184]
